# MobileNet

In [ ]:
# Import OS module
import os

# Imports
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks, layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Notebook magic
%matplotlib inline

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)
grid_shape = (1, image_size, image_size, 3)

batch_size = 32

In [ ]:
train_dir = '/content/drive/MyDrive/Copy of New_Training_resized'


#Define augmentations for train dataset and read the images
train_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0,
    # Filling for W/H shift
    fill_mode="nearest",
    # Width and Height shift
    width_shift_range=0.2,
    height_shift_range=0.2,
    # Random zooms
    zoom_range=0.2,
    # Random Shearing aug
    shear_range=0.2,
)

# Read data from directory
train_data = train_aug.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 2163 images belonging to 4 classes.


In [ ]:
# Get the list of categories in training data
cats = list(train_data.class_indices.keys())

In [ ]:
test_dir = '/content/drive/MyDrive/New_Validation_resized'
# Augmentations for test data
test_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 575 images belonging to 4 classes.


In [ ]:
# Load the base model
mbnet_v2 = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape
)

# Stop from being trainable
mbnet_v2.trainable = False

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
# Define the layers
inputs = keras.Input(shape=input_shape)

# Get the layer
x = mbnet_v2(inputs, training = False)

# Stack layers further
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(cats), activation="softmax")(x)

# Combine the model
model = Model(inputs=inputs, outputs=x)

# Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 2263108 (8.63 MB)
Trainable params: 5124 (20.02

In [ ]:
# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Define callbacks to use
early_stopping_cb = callbacks.EarlyStopping(monitor="loss", patience=3)

In [ ]:
# Num epochs
epochs = 30

# Train model
history = model.fit(
    train_data,
    epochs=epochs,
    steps_per_epoch=65,
    callbacks=[early_stopping_cb]
)

Epoch 1/30
65/65 [==============================] - 1328s 20s/step - loss: 1.1758 - accuracy: 0.5046
Epoch 2/30
65/65 [==============================] - 118s 2s/step - loss: 1.0025 - accuracy: 0.5947
Epoch 3/30
65/65 [==============================] - 102s 2s/step - loss: 0.9260 - accuracy: 0.6425
Epoch 4/30
65/65 [==============================] - 102s 2s/step - loss: 0.8985 - accuracy: 0.6459
Epoch 5/30
65/65 [==============================] - 103s 2s/step - loss: 0.8715 - accuracy: 0.6613
Epoch 6/30
65/65 [==============================] - 105s 2s/step - loss: 0.8422 - accuracy: 0.6837
Epoch 7/30
65/65 [==============================] - 103s 2s/step - loss: 0.8491 - accuracy: 0.6613
Epoch 8/30
65/65 [==============================] - 103s 2s/step - loss: 0.8460 - accuracy: 0.6672
Epoch 9/30
65/65 [==============================] - 102s 2s/step - loss: 0.8186 - accuracy: 0.6788
Epoch 10/30
65/65 [==============================] - 103s 2s/step - loss: 0.8221 - accuracy: 0.6744
Epoch 1

In [ ]:
model.evaluate(test_data)

18/18 [==============================] - 335s 20s/step - loss: 1.5538 - accuracy: 0.4365


[1.5538313388824463, 0.4365217387676239]